In [ ]:
%pip install -U langchain_community langchain-openai langchain-ollama langchain langgraph bs4

In [ ]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/concepts/lcel/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(concatenated_content)


## 

## 测试写代码

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
    Here is a full set of LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user 
    question based on the above provided documentation. Ensure any code you provide can be executed \n 
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


expt_llm = "qwen2.5:7b"
llm = ChatOllama(temperature=0, model=expt_llm)

# 将输出结果结构化为对象
code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(code)
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain_oai.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
# 美化输出
print("【问题解析】\n")
print(solution.prefix)
print("\n" + "="*40 + "\n")
print("【所需导入】\n")
print(solution.imports)
print("\n" + "="*40 + "\n")
print("【完整代码】\n")
print(solution.code)

【问题解析】

To build a Retrieval-Augmented Generation (RAG) chain using LangChain Expression Language (LCEL), you need to combine several Runnables that work together to retrieve relevant documents and generate text based on those documents. Here's a step-by-step guide on how to do this:


【所需导入】

from langchain.chains import RetrievalAugmentedGenerationChain
from langchain.embeddings import Embeddings
from langchain.vectorstores import VectorStore


【完整代码】

```python
# Step 1: Define the embeddings and vector store
embeddings = Embeddings()
vector_store = VectorStore(embeddings=embeddings)

# Step 2: Create a RetrievalAugmentedGenerationChain
rag_chain = RetrievalAugmentedGenerationChain(
    retriever=vector_store.as_retriever(),
    generator=generator,  # Replace with your LLM or chat model
    prompt=prompt_template,  # Replace with your prompt template
)

# Step 3: Invoke the chain with input
input_text = 'What is the capital of France?'  # Example query
output = rag_chain.invoke(inp

## 测试纠正代码


In [6]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

### Anthropic

# Prompt to enforce tool use
code_gen_prompt_ollama = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user  question based on the \n 
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    Invoke the code tool to structure the output correctly. </instructions> \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# LLM
expt_llm = "qwen2.5:7b"  # 或 "llama3"、"phi3" 等
llm = ChatOllama(
    model=expt_llm,
    temperature=0,
)
structured_llm_ollama = llm.with_structured_output(code, include_raw=True)


# Optional: Check for errors in case tool use is flaky
def check_claude_output(tool_output):
    """Check for parse error or failure to call the tool"""

    # Error with parsing
    if tool_output["parsing_error"]:
        # Report back output and parsing errors
        print("Parsing error!")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )

    # Tool was not invoked
    elif not tool_output["parsed"]:
        print("Failed to invoke tool!")
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    return tool_output


# Chain with output check
code_chain_claude_raw = (
    code_gen_prompt_ollama | structured_llm_ollama | check_claude_output
)


def insert_errors(inputs):
    """Insert errors for tool parsing in the messages"""

    # Get errors
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool.",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


# This will be run as a fallback chain
fallback_chain = insert_errors | code_chain_claude_raw
N = 3  # Max re-tries
code_gen_chain_re_try = code_chain_claude_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)


def parse_output(solution):
    """When we add 'include_raw=True' to structured output,
    it will return a dict w 'raw', 'parsed', 'parsing_error'."""

    return solution["parsed"]


# Optional: With re-try to correct for failure to invoke tool
code_gen_chain = code_gen_chain_re_try | parse_output

# No re-try
code_gen_chain = code_gen_prompt_ollama | structured_llm_ollama | parse_output

In [8]:
# Test
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
# 美化输出
print("【问题解析】\n")
print(solution.prefix)
print("\n" + "="*40 + "\n")
print("【所需导入】\n")
print(solution.imports)
print("\n" + "="*40 + "\n")
print("【完整代码】\n")
print(solution.code)

【问题解析】

To build a Retrieval-Augmented Generation (RAG) chain using LangChain Expression Language (LCEL), you can follow these steps. This example will demonstrate how to create a simple RAG chain that retrieves documents from a vector store and uses them to augment the input before passing it through an LLM.


【所需导入】

from langchain.chains import RetrievalQA
from langchain.embeddings import Embeddings
from langchain.vectorstores import VectorStore


【完整代码】

class SimpleRagChain(RetrievalQA):
    def __init__(self, embeddings: Embeddings, vectorstore: VectorStore, llm, **kwargs):
        super().__init__(embeddings=embeddings, retriever=vectorstore.as_retriever(), llm=llm, **kwargs)

# Example usage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI

embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.load_local('path/to/vectorstore', embeddings)
llm = OpenAI()
rag_chain = SimpleRagChain(embeddings=embe